In [1]:
import arviz as az
import pymc as pm
from pymc.math import log, sqr

%load_ext lab_black

# Zero-tricks and Custom Likelihoods*

## Zero-trick Jeremy

This introduces the "zero trick", which I'm still not sure if we'll ever need. Including for completeness' sake.

Adapted from [Unit 6: zerotrickjeremy.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit6/zerotrickjeremy.odc).

There's a running example in the lectures about Jeremy testing his IQ. At some point I will track all those down and add links here, but for now I'm just going to port the code.

I'm not sure what's going on when the professor sets ```z1``` as both a deterministic and random variable. I'll need to test some things out in BUGS once the Citrix virtual machines are back online. For now, here's a first pass at recreating the model, where I interpret the ```z <- 0``` as feeding an observation of zero to the variable.

That said, I don't think we will ever need to use the zeros or ones tricks in the homeworks. If we do I will definitely expand this page.

In [2]:
y = 98
μ = 110
σ = 8.944272
τ = 10.954451
constant = 1000  # can't let lambda be lower than zero

inits = {"θ": 100}

In [3]:
with pm.Model() as m:
    θ = pm.Flat("θ")

    λ1 = pm.Deterministic("λ1", log(σ) + 0.5 * sqr(((y - θ) / σ)) + constant)
    λ2 = pm.Deterministic("λ2", log(τ) + 0.5 * sqr(((θ - μ) / τ)) + constant)

    z1 = pm.Poisson("z1", λ1, observed=0)
    z2 = pm.Poisson("z2", λ2, observed=0)

    trace = pm.sample(5000, tune=1000, initvals=inits, target_accept=0.88)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [θ]
Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 9 seconds.
Chain <xarray.DataArray 'chain' ()>
array(0)
Coordinates:
    chain    int64 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain <xarray.DataArray 'chain' ()>
array(1)
Coordinates:
    chain    int64 1 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain <xarray.DataArray 'chain' ()>
array(2)
Coordinates:
    chain    int64 2 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain <xarray.DataArray 'chain' ()>
array(3)
Coordinates:
    chain    int64 3 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameteriz

In [4]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
θ,102.781,6.932,89.442,116.391,0.057,0.040,14915.0,22114.0,1.0
λ1,1002.634,0.584,1002.191,1003.836,0.004,0.003,17998.0,22701.0,1.0
λ2,1002.811,0.505,1002.394,1003.846,0.004,0.003,16323.0,22175.0,1.0


Again, it's not clear to me what BUGS is doing, but these PyMC results are almost exactly the same as the professor's results so this must be close. I have also tried passing vectors of zeros to each ```z```, putting more weight on the zero "observations." This ended up reducing the credible interval and standard deviation of theta. 

I found [this page](http://www.medicine.mcgill.ca/epidemiology/Joseph/courses/common/Tricks.html) that briefly mentions the same trick. They note that "... this method can be very inefficient and give a very high MC error."

Statistical Rethinking 12.2 talks about zero-inflation, may need to give that a read.


In [5]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Sat Mar 18 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

pytensor: 2.10.1

arviz     : 0.14.0
numpy     : 1.24.2
matplotlib: 3.6.3
pymc      : 5.1.1

